In [1]:
# Import necessary modules
import pandas as pd
import numpy as np
import glob
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
import os 


In [2]:
# Select station
station = 'HTM'

# Define variable list to loop over later
fluxfilelist = ['nep', 'fire', 'ocean', 'anthropogenic']
fluxnamelist = ['nep', 'fire', 'ocean', 'combustion']
fluxdir = '/projects/0/ctdas/awoude/NRT/ICOS_OUTPUT/'

# Simulation length
sim_length = 240

In [3]:
# Define footprint file list
filepath = '/gpfs/scratch1/shared/dkivits/STILT/footprints/'
outpath = '/gpfs/scratch1/shared/awoude/STILT/footprints_multiplied/'

# Create directory if it does not exist
if not os.path.exists(outpath):
    os.makedirs(outpath)

In [5]:
csv_files = sorted(glob.glob(filepath+'footprint_' + station + '*.csv'))

timestr_start = csv_files[0][-38:-25]
timestr_end = csv_files[-1][-38:-25]

In [6]:
fp_range_start = datetime.strptime(timestr_start, '%Yx%mx%dx%H')- timedelta(hours=sim_length)
fp_range_end = datetime.strptime(timestr_end, '%Yx%mx%dx%H') 

In [7]:
# Extract all unique months between start and end time
mons = pd.date_range(fp_range_start, fp_range_end, freq='D').strftime("%Y%m").unique().tolist()

In [8]:
# Loop over all CTE-HR flux variables
fluxstring = []
for var in fluxfilelist:
    for mon in mons:
        # Define which CTE-HR flux files to loop over
        fluxstring += sorted(glob.glob(fluxdir + var + '.' + mon + '.nc'))

In [9]:
# Open all files in fluxstring as xr_mfdataset
ds = xr.open_mfdataset(fluxstring, combine='by_coords')
file = csv_files[0]
# Read footprint file
fp_df = pd.read_csv(file, header=0)

# Up untill here, it is very similar to what you did.

In [43]:
# First, find the time indices into the xarray dataset
# We do this by 
# 1. getting the difference between the first time in the xr dataset and the footrpint file
# 2. calculating this time difference to an hour
# 3. calculating the hours backwards in the df to indices in the xr dataset
time_diff = (fp_range_start - datetime.strptime(mons[0], '%Y%m'))
hours_since_start_of_ds = time_diff.seconds // 3600 + time_diff.days * 24
hours_into_file = hours_since_start_of_ds - fp_df.times.values -1

In [50]:

# Select the fluxes where there is footprint influence
# and then multiply with the footprint
# For each of the important fluxes (bio, ocean, fire, anthropogenic)
sum([(ds[v].values[hours_into_file,
                  fp_df.index_y - 1,
                  fp_df.index_x - 1] * fp_df['values']).sum()
     for v in fluxnamelist])

-1.2683007419525867e-05